In [3]:
!pip install groq
import requests
from bs4 import BeautifulSoup
import pandas as pd
import os
from openai import OpenAI
from google.colab import userdata
from groq import Groq

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 108.8/108.8 kB 4.7 MB/s eta 0:00:00


In [4]:
client = Groq(
    api_key=userdata.get("GROQ_API_KEY"),
)

In [5]:
import requests
from bs4 import BeautifulSoup


# Making a GET request
r = requests.get('https://abcnews.go.com/US/wireStory/new-mexico-seeks-record-478m-fine-excess-air-116542841')

# check status code for response received
# success code - 200
print(r)

# Parsing the HTML
soup = BeautifulSoup(r.content, 'html.parser')
content = soup.find_all("p")

<Response [200]>


In [6]:
content

[<p class="jxTEW Poyse uieav lqtkC HkWF HfYhe kGyAC"><span class="gtOSm FbbUW tUtYa vOCwz EQwFq yCufu eEak Qmvg nyTIa SRXVc vzLa jgBfc WXDas CiUCW kqbG zrdEG txGfn ygKVe BbezD UOtxr CVfpq xijV soGRS XgdC sEIlf daWqJ">New Mexico environmental regulators have issued a $47.8 million fine on allegations of excess air pollution at a natural gas processing facility in a prolific oil production region</span></p>,
 <p class="EkqkG IGXmU nlgHS yuUao MvWXB TjIXL aGjvy ebVHC"><span class="oyrPY qlwaB AGxeB">SANTA FE, N.M. -- </span>New Mexico environmental regulators on Friday issued a $47.8 million fine on allegations of excess air pollution at a natural gas processing facility in a prolific oil production region near the Texas state line.</p>,
 <p class="EkqkG IGXmU nlgHS yuUao lqtkC TjIXL aGjvy">The state Environment Department issued the sanctions including a cease and desist order against Houston-based Targa Resources at its processing plant near Jal, New Mexico, alleging permit violations a

In [7]:
article_text=" ".join([x.text for x in content])

In [25]:
# Assuming `article_text` is a string containing the article content

prompt = f"""you are an expert financial analyst.

Based on the text in the article below, return a list of stock tickers that are directly and indirectly related to the article.

<article_text>
  {article_text}
</article_text>

return your response in JSON format like this example:

{{
  "response":[
    {{
      "ticker":"AAPL",
      "explanation": "<explanation of why this ticker is relevant to the article>"
      "sentiment": <number from 0-10 on how positive or negative the topic on the article will effect for the company>
      "sector": "<sector the company belong to>",
      "industry": "<industry the company belong to>",
      "market_cap": "<market capitalization of the company>"
    }},
    {{
      "ticker":"GOOGL",
      "explanation": "<explanation of why this ticker is relevant to the article>"
      "sentiment": <number from 0-10 on how positive or negative the topic on the article will effect for the company>
      "sector": "<sector the company belong to>",
      "industry": "<industry the company belong to>",
      "market_cap": "<market capitalization of the company>"

    }}
  ]
}}
"""

chat_completion = client.chat.completions.create(
    model="llama-3.1-70b-versatile",
    messages=[
        # {"role": "system", "content": system_prompt},
        {"role": "user", "content": prompt}
    ],
    response_format={"type": "json_object"},
)

# Print the response from the model
response = chat_completion.choices[0].message.content
print(response)


{
   "response":[
      {
         "ticker":"TRGP",
         "explanation":"Targa Resources is mentioned as the company receiving a $47.8 million fine for excess air pollution at one of its processing facilities.",
         "sentiment":2,
         "sector":"Energy",
         "industry":"Oil and Gas",
         "market_cap":"23.48B"
      },
      {
         "ticker":"ENB",
         "explanation":"Enbridge Inc. is a company involved in oil and gas pipeline transmission and may be indirectly affected by increased regulations on oil and gas companies in New Mexico.",
         "sentiment":3,
         "sector":"Energy",
         "industry":"Oil and Gas",
         "market_cap":"120.81B"
      },
      {
         "ticker":"PAA",
         "explanation":"Plains All American Pipeline is a company involved in oil and gas pipeline transmission and may be indirectly affected by increased regulations on oil and gas companies in New Mexico.",
         "sentiment":3,
         "sector":"Energy",
       

## **Using Pinecone + WebScraping Together**



In [27]:
! pip install yfinance langchain_pinecone openai python-dotenv langchain-community sentence_transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 33.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 39.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 44.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 409.7/409.7 kB 28.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.8/244.8 kB 18.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.5/49.5 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.4/85.4 kB 7.2 MB/s eta 0:00:00
  Attempting uninstall: aiohttp
    Found existing installation: aiohttp 3.11.9
    Uninstalling aiohttp-3.11.9:
      Successfully uninstalled aiohttp-3.11.9
  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 0.3.21
    Uninstalling langchain-core-0.3.21:
      Successfully uninstalled langchain-core-0.3.21
  Attempting uninstall: langchain
    Found existing installation: langchain 0.3.9
    Uninstalling langc

In [28]:
from langchain_pinecone import PineconeVectorStore
from openai import OpenAI
import dotenv
import json
import yfinance as yf
import concurrent.futures
from langchain_community.embeddings import HuggingFaceEmbeddings
from google.colab import userdata
from langchain.schema import Document
from sklearn.metrics.pairwise import cosine_similarity
from sentence_transformers import SentenceTransformer
from pinecone import Pinecone
import numpy as np
import requests
import os

In [29]:
def get_huggingface_embeddings(text, model_name="sentence-transformers/all-mpnet-base-v2"):
    """
    Generates embeddings for the given text using a specified Hugging Face model.

    Args:
        text (str): The input text to generate embeddings for.
        model_name (str): The name of the Hugging Face model to use.
                          Defaults to "sentence-transformers/all-mpnet-base-v2".

    Returns:
        np.ndarray: The generated embeddings as a NumPy array.
    """
    model = SentenceTransformer(model_name)
    return model.encode(text)

In [30]:
query = article_text

In [31]:
raw_query_embedding = get_huggingface_embeddings(query)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [32]:
system_prompt = f"""You are an expert at providing answers about stocks. Please answer my question provided.
"""

In [33]:
pinecone_api_key = userdata.get("PINECONE_API_KEY")
os.environ['PINECONE_API_KEY'] = pinecone_api_key

index_name = "stocks"
namespace = "stock-descriptions"

hf_embeddings = HuggingFaceEmbeddings()
vectorstore = PineconeVectorStore(index_name=index_name, embedding=hf_embeddings)

<ipython-input-33-8d5c0f356cf6>:7: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  hf_embeddings = HuggingFaceEmbeddings()
<ipython-input-33-8d5c0f356cf6>:7: LangChainDeprecationWarning: Default values for HuggingFaceEmbeddings.model_name were deprecated in LangChain 0.2.16 and will be removed in 0.4.0. Explicitly pass a model_name to the HuggingFaceEmbeddings constructor instead.
  hf_embeddings = HuggingFaceEmbeddings()


In [34]:
# Initialize Pinecone
pc = Pinecone(api_key=userdata.get("PINECONE_API_KEY"),)

# Connect to your Pinecone index
pinecone_index = pc.Index(index_name)

In [35]:
top_matches = pinecone_index.query(vector=raw_query_embedding.tolist(), top_k=10, include_metadata=True, namespace=namespace)

In [45]:
top_matches

{'matches': [{'id': 'f196b911-a71e-4afc-8696-7a5639eb44df',
              'metadata': {'Business Summary': 'Targa Resources Corp., '
                                               'together with its subsidiary, '
                                               'Targa Resources Partners LP, '
                                               'owns, operates, acquires, and '
                                               'develops a portfolio of '
                                               'complementary domestic '
                                               'midstream infrastructure '
                                               'assets in North America. It '
                                               'operates in two segments, '
                                               'Gathering and Processing, and '
                                               'Logistics and Transportation. '
                                               'The company is involved in '
               

In [52]:
augumented_query = f"""
You are an expert financial analyst.

Based on the provided company information and a news article, return a list of stock tickers that are directly and indirectly related to the company and the article's topic.

**Companies Information:**

{top_matches}

Use these companies info as an releveant info top matches
**News Article:**

<article_text>
  {article_text}
</article_text>


**Instructions:**

1. Identify keywords and concepts from the company information and news article.
2. Leverage your financial expertise to find related companies and their stock tickers.
3. Prioritize companies with a direct or strong indirect relationship.
4. Provide clear explanations for the relevance of each ticker.
5. Estimate the sentiment (0-10) representing the potential impact (positive or negative) of the article's topic on each company.
6. Include the sector, industry, and market capitalization for each company.


**Response Format:**

Return your response in JSON format:


{{
  "response":[
    {{
      "ticker":"AAPL",
      "explanation": "<explanation of why this ticker is relevant to the article>",
      "sentiment": <number from 0-10>,
      "sector": "<sector>",
      "industry": "<industry>",
      "market_cap": "<market capitalization>"
      "volume":integer
    }},
    {{
      "ticker":"GOOGL",
      "explanation": "<explanation of why this ticker is relevant to the article>",
      "sentiment": <number from 0-10>,
      "sector": "<sector>",
      "industry": "<industry>",
      "market_cap": "<market capitalization>"
       "volume":integer
    }}
  ]
}}
"""

In [53]:
augumented_query

'\nYou are an expert financial analyst.\n\nBased on the provided company information and a news article, return a list of stock tickers that are directly and indirectly related to the company and the article\'s topic.\n\n**Companies Information:**\n\n{\'matches\': [{\'id\': \'f196b911-a71e-4afc-8696-7a5639eb44df\',\n              \'metadata\': {\'Business Summary\': \'Targa Resources Corp., \'\n                                               \'together with its subsidiary, \'\n                                               \'Targa Resources Partners LP, \'\n                                               \'owns, operates, acquires, and \'\n                                               \'develops a portfolio of \'\n                                               \'complementary domestic \'\n                                               \'midstream infrastructure \'\n                                               \'assets in North America. It \'\n                                          

In [55]:
chat_completion = client.chat.completions.create(
    model="llama-3.1-8b-instant",
    messages=[
        # {"role": "system", "content": system_prompt},
        {"role": "user", "content": augumented_query}
    ],
    response_format={"type": "json_object"},
)

# Print the response from the model
response = chat_completion.choices[0].message.content
print(response)

{
   "response": [
    {
      "ticker": "TRGP",
      "explanation": "Targa Resources Corp. is a natural gas processing company with a facility in New Mexico, which was fined for excess air pollution. This has significant implications for the company's operations and finances.",
      "sentiment": -8,
      "sector": "Energy",
      "industry": "Oil & Gas Midstream",
      "market_cap": 10324436000,
      "volume": 1294000
    },
    {
      "ticker": "CLNE",
      "explanation": "Clean Energy Fuels Corp. is a company that provides RNG, CNG, and LNG for transportation, which is affected by natural gas processing and emissions regulations.",
      "sentiment": -6,
      "sector": "Transportation",
      "industry": "Energy",
      "market_cap": 23440000,
      "volume": 1050908
    },
    {
      "ticker": "DTM",
      "explanation": "DT Midstream, Inc. provides integrated natural gas services, including transportation and storage, which are relevant to the article's topic of natural g